In [14]:
import os
import glob
import subprocess
from osgeo import gdal, osr

In [15]:
# Use absolute paths
raw_lidar_dir = '/Users/kdoherty/calibrate_rap/data/raster/lidar_raw/chms/{YEAR}'
processed_lidar_dir = '/Users/kdoherty/calibrate_rap/data/raster/lidar_processed/chms/'

years = [2008, 2009, 2012, 2016, 2019, 2020]


for year in years:
    print(f"Processing year: {year}")
    # Define directories for raw and processed lidar data for this year
    raw_dir = raw_lidar_dir.format(YEAR=year)
    processed_lidar_dir
    os.makedirs(processed_lidar_dir, exist_ok=True)

    # List all TIF files in the raw directory (case insensitive)
    tif_files = []
    for ext in ['*.tif', '*.TIF', '*.tiff', '*.TIFF']:
        tif_files.extend(glob.glob(os.path.join(raw_dir, ext)))
    
    if not tif_files:
        print(f"No TIF files found for year {year} in {raw_dir}. Skipping.")
        print(f"Searched path: {os.path.join(raw_dir, '*.tif')}")
        print(f"Directory exists: {os.path.exists(raw_dir)}")
        if os.path.exists(raw_dir):
            print(f"Directory contents: {os.listdir(raw_dir)}")
        continue

    # Create a temporary directory for any reprojected files for this year
    temp_year_dir = os.path.join("temp", str(year))
    os.makedirs(temp_year_dir, exist_ok=True)

    reprojected_files = []
    for tif in tif_files:
        ds = gdal.Open(tif)
        if ds is None:
            print(f"Warning: Could not open {tif}. Skipping.")
            continue
        proj_wkt = ds.GetProjection()
        srs = osr.SpatialReference()
        srs.ImportFromWkt(proj_wkt)
        epsg_code = srs.GetAttrValue("AUTHORITY", 1)
        # Get the nodata value from the first band (if set)
        band = ds.GetRasterBand(1)
        nodata_val = band.GetNoDataValue()

        # If not already in EPSG:6514, reproject the file
        if epsg_code != "6514":
            reproj_path = os.path.join(temp_year_dir, "reproj_" + os.path.basename(tif))
            warp_cmd = ["gdalwarp", "-t_srs", "EPSG:6514", "-overwrite"]
            if nodata_val is not None:
                warp_cmd.extend(["-dstnodata", str(nodata_val)])
            warp_cmd.extend([tif, reproj_path])
            print(f"Reprojecting {tif} to EPSG:6514 as {reproj_path}")
            subprocess.run(warp_cmd, check=True)
            reprojected_files.append(reproj_path)
        else:
            reprojected_files.append(tif)

    if not reprojected_files:
        print(f"No valid TIF files to process for year {year}.")
        continue

    # Build a temporary VRT for all TIFs of this year
    temp_vrt = os.path.join("temp", f"{year}.vrt")
    os.makedirs("temp", exist_ok=True)
    vrt_cmd = ["gdalbuildvrt", "-allow_projection_difference", "-resolution", "highest", temp_vrt] + reprojected_files
    print(f"Creating VRT for year {year} at {temp_vrt}")
    subprocess.run(vrt_cmd, check=True)

    # Translate the VRT to a final CHM TIF, preserving nodata values.
    output_tif = os.path.join(processed_lidar_dir, f'{year}.tif')
    # Optionally, get the nodata value from the first file in our list
    sample_ds = gdal.Open(reprojected_files[0])
    sample_band = sample_ds.GetRasterBand(1)
    sample_nodata = sample_band.GetNoDataValue()
    
    translate_cmd = ["gdal_translate"]
    if sample_nodata is not None:
        translate_cmd.extend(["-a_nodata", str(sample_nodata)])
    translate_cmd.extend([temp_vrt, output_tif])
    print(f"Translating VRT to final TIF for year {year}: {output_tif}")
    subprocess.run(translate_cmd, check=True)

    # Clean up temporary files
    import shutil
    if os.path.exists(temp_year_dir):
        shutil.rmtree(temp_year_dir)
    if os.path.exists("temp"):
        shutil.rmtree("temp")

    print(f"Finished processing year {year}. Output saved to {output_tif}\n")


Processing year: 2008
Creating VRT for year 2008 at temp/2008.vrt
0...10...20...30...40...50...60...70...80...90...100 - done.
Translating VRT to final TIF for year 2008: /Users/kdoherty/calibrate_rap/data/raster/lidar_processed/chms/2008.tif
Input file size is 16783, 18209
0...10...20...30...40...50...60...70...80...90...100 - done.
Finished processing year 2008. Output saved to /Users/kdoherty/calibrate_rap/data/raster/lidar_processed/chms/2008.tif

Processing year: 2009
Creating VRT for year 2009 at temp/2009.vrt
0...10...20...30...40...50...60...70...80...90...100 - done.
Translating VRT to final TIF for year 2009: /Users/kdoherty/calibrate_rap/data/raster/lidar_processed/chms/2009.tif
Input file size is 7009, 14227
0...10...20...30...40...50...60...70...80...90...100 - done.
Finished processing year 2009. Output saved to /Users/kdoherty/calibrate_rap/data/raster/lidar_processed/chms/2009.tif

Processing year: 2012
Creating VRT for year 2012 at temp/2012.vrt
0...10...20...30...40..